In [3]:
import nltk
nltk.download("stopwords")
#stopword corpus to remove stopwords from sentences

[nltk_data] Error loading stopwords: HTTP Error 405: Not allowed.


False

In [4]:
import numpy as np
import pandas as pd
document1=pd.read_csv("spam.csv")

In [5]:
document1.head()

#data from kaggle spam-ham competition


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
from sklearn.cross_validation import train_test_split
document2 = np.array(document1)  #convert array to numpy type array
#split data into test train
x_train ,x_test = train_test_split(document2,test_size=0.3)   

C:\Users\propylon\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
doc2=pd.DataFrame(x_train)
doc3=pd.DataFrame(x_test)
doc2.head()

,0,1,2,3,4
0,ham,There is a first time for everything :),NaN,NaN,NaN
1,spam,Not heard from U4 a while. Call 4 rude chat pr...,NaN,NaN,NaN
2,ham,God created gap btwn ur fingers so dat sum1 vr...,NaN,NaN,NaN
3,ham,I am not having her number sir,NaN,NaN,NaN
4,ham,Ummma.will call after check in.our life will b...,NaN,NaN,NaN


In [8]:

from nltk.tokenize import RegexpTokenizer
#converting sentences for every input in form of tokens , every word would represent a token
tokenizer = RegexpTokenizer(r'\w+') #removes punctuations
spam=[]
ham=[]
for index, row in doc2.iterrows():
    if row[0]=='spam':
        #converted all the words to lowercase
        spam.append(tokenizer.tokenize(row[1].lower())) #spam list of sentences
    if row[0]=='ham':
        ham.append(tokenizer.tokenize(row[1].lower())) #ham 
test_dataset=[]
test_label=[]
for index, row in doc3.iterrows():
    test_dataset.append(tokenizer.tokenize(row[1].lower())) #test dataset
    test_label.append(tokenizer.tokenize(row[0].lower())) #test labels

In [9]:
from nltk.corpus import stopwords
#stopwords corpus which we downloaded initially
def flatted_list(texted):
    return [item for sublist in texted for item in sublist]
# creating spam and ham list of words appearing in the respective inputs
spam_flat=flatted_list(spam)
ham_flat=flatted_list(ham)
#creating flat list of spam and ham

#removing stopwords from spam and ham list
spam_flat1=[word for word in spam_flat if word not in stopwords.words('english')] 
ham_flat1=[word for word in ham_flat if word not in stopwords.words('english')]
print len(spam_flat),len(spam_flat1)


C:\Users\propylon\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()
C:\Users\propylon\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if sys.path[0] == '':


13482 9713


In [10]:

total=len(spam)+len(ham) #total number of training sentences
#prior probabilities
spam_prob=len(spam)/float(total) 
ham_prob=len(ham)/float(total) 

import nltk 
from nltk.probability import FreqDist

#can also create bigram, ngram words # but used unigram from simplification 
#Freqdist counts a particular word appearing in the corpus

def unigram(word):
    # Write your code here.
    fdist1=FreqDist(word)
    return fdist1
    


In [11]:
spam_uni=unigram(spam_flat1) #calculate count of any word/ similarly can be done for bigrams/ngrams
#print spam_uni[('win')]
ham_uni=unigram(ham_flat1)
#print ham_uni[(['happy'])]

In [12]:

predict_labels=[]
for i,sentence in enumerate(test_dataset):
    ham_bayes=ham_prob #prior prob
    spam_bayes=spam_prob
    sentence1=[word for word in sentence if word not in stopwords.words('english')]
    #removing stopwords from test data
    for k,word in enumerate(sentence1):
        #laplacian and prob(word/spam) 
        spam_pro=float(spam_uni[word]+1)/float(len(spam_flat1)+1)
        #prob(word/ham)
        ham_pro=float(ham_uni[word]+1)/float(len(ham_flat1)+1)
        #naive bayes calculation , gets multiplied with every loop 
        spam_bayes=spam_bayes*spam_pro
        ham_bayes=ham_bayes*ham_pro
    #normalizing the values    
    normalized_spam=float(spam_bayes)/float(spam_bayes+ham_bayes)
    normalized_ham=float(ham_bayes)/float(spam_bayes+ham_bayes)
    #predicting whether the sentence is spam or ham, based on the normalized values
    if normalized_spam>normalized_ham:
        predict_labels.append('spam')
    else:
        predict_labels.append('ham')

C:\Users\propylon\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [13]:
acc=0
for i1 in range(len(test_label)):
    if test_label[i1][0]==predict_labels[i1]:
        acc=acc+1
float(acc)/float(i1)

0.9539198084979055